# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
#Passo 1 : Entrar no Google

from selenium import webdriver
from selenium.webdriver.common.by import By # Localizar elementos( os itens do site)
from selenium.webdriver.common.keys import Keys # Permite clicar teclas no  teclado


navegador = webdriver.Chrome()

navegador.get("https://www.google.com.br/")

#Passo 2 : Pesquisar a cotação do dolar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


#Passo 3 : Pegar a cotação do dolar

cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

#Passo 4 : Pegar a cotação do euro

navegador.get('https://www.google.com.br/')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

#Passo 5 : Pegar a cotação do ouro

navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20299%2C71.')

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(",",".")

print(cotacao_ouro)

#PAsso 6 : Atualizar minha base de dados com as novas cotação

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

display(tabela)








5.0577
5.260260885
293.48


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [2]:
!pip install selenium


### Agora vamos atualiza a nossa base de preços com as novas cotações

In [5]:
# atualizar a cotação de acordo com a moeda correspondente
#ouro

tabela.loc[tabela["Moeda"] == "Dólar","Cotação"] = float(cotacao_dolar)

#euro

tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

#ouro

tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

display(tabela)



#ataulizar o preço de compra -> Preço original x cotação


tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#atualizar o preço de venda -> Preço de compra x margem

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.057700,5057.649423,1.40,7080.709192
1,Carro Renault,4500.00,Euro,5.260261,23671.173983,2.00,47342.347965
2,Notebook Dell,899.99,Dólar,5.057700,4551.879423,1.70,7738.195019
3,IPhone,799.00,Dólar,5.057700,4041.102300,1.70,6869.873910
4,Carro Fiat,3000.00,Euro,5.260261,15780.782655,1.90,29983.487045
5,Celular Xiaomi,480.48,Dólar,5.057700,2430.123696,2.00,4860.247392
6,Joia 20g,20.00,Ouro,293.480000,5869.600000,1.15,6750.040000


- Importando a base de dados

- Atualizando os preços e o cálculo do Preço Final

### Agora vamos exportar a nova base de preços atualizada

In [4]:
tabela.to_excel("Produto Novo.xlsx", index=False)